# Set up libraries, import packages, set seed

In [ ]:
!pip install py_vncorenlp
!pip install transformers
!pip install evaluate
!pip install accuracy
!pip install accelerate -U

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 16.3 MB/s eta 0:00:00
  Created wheel for py_vncorenlp: filename=py_vncorenlp-0.1.4-py3-none-any.whl size=4306 sha256=27c02f46814c3ab61929f9486696738a30520a7575b235bf244b47ce54d5fd95
  Stored in directory: /root/.cache/pip/wheels/d5/d9/bf/62632cdb007c702a0664091e92a0bb1f18a2fcecbe962d9827
Successfully built py_vncorenlp
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 32.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 25.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 68.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 43.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 28.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import torch
from transformers.file_utils import is_tf_available, is_torch_available, is_torch_tpu_available
from transformers import AutoModelForSequenceClassification, AutoTokenizer, TrainingArguments, Trainer
import numpy as np
from sklearn.model_selection import train_test_split
import ast
import random
import torch.optim as optim

In [ ]:
import evaluate

accuracy = evaluate.load("accuracy")

In [ ]:
import py_vncorenlp
py_vncorenlp.download_model(save_dir='/content/vncorenlp')

segmenter = py_vncorenlp.VnCoreNLP(annotators=["wseg"],save_dir=r"/content/vncorenlp")

In [ ]:
def set_seed(seed: int):
    """
    Helper function for reproducible behavior to set the seed in ``random``, ``numpy``, ``torch`` and/or ``tf`` (if
    installed).

    Args:
        seed (:obj:`int`): The seed to set.
    """
    random.seed(seed)
    np.random.seed(seed)
    if is_torch_available():
        torch.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
        # ^^ safe to call this function even if cuda is not available
    if is_tf_available():
        import tensorflow as tf

        tf.random.set_seed(seed)

set_seed(1)

# Get data
Data lấy file raw github nhma theo đường dẫn tới file raw xong bỏ đường dẫn mới vào raw_url mới dùng được ko biết sao nhma hình như do url time out nên ko tìm thấy

In [ ]:
url_to_raw = 'https://github.com/hallie304/VLSP23-Comparative-Opinion-Mining/blob/main/data/total.csv?raw=true'
raw_url ="https://raw.githubusercontent.com/hallie304/VLSP23-Comparative-Opinion-Mining/main/data/total.csv?token=GHSAT0AAAAAACHTULZCJDQLSQ35KSRYJAMCZJG2IJQ"
df = pd.read_csv(raw_url, index_col = 0)
comparative = df
# comparative.label.iloc[815]

#Set up automodel

In [ ]:
content_list = comparative[["content","comparative"]]

In [ ]:
id2label = {0:"Not Comparative", 1:"Comparative"}
label2id = {"Not Comparative":0,"Comparative":1}

In [ ]:
phobert = AutoModelForSequenceClassification.from_pretrained("vinai/phobert-base-v2", num_labels=2, id2label=id2label, label2id=label2id)
tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base-v2")

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at vinai/phobert-base-v2 and are newly initialized: ['classifier.out_proj.weight', 'classifier.out_proj.bias', 'classifier.dense.bias', 'classifier.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


#Data splitting and tokenizing

In [ ]:
content_list["content"] = content_list["content"].map(lambda x: segmenter.word_segment(x)[0])

<ipython-input-37-b8aaaf5e4702>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  content_list["content"] = content_list["content"].map(lambda x: segmenter.word_segment(x)[0])


In [ ]:
content_list["content"][1]

'iPhone 14 được ra_mắt vào 09 / 2022 , được đánh_giá là chỉ nâng_cấp nhẹ về phần_cứng cũng như phần_mềm so với thế_hệ trước .'

In [ ]:
content_list["comparative"][1]

1

In [ ]:
dct = {'content':[],'comparative':[]}
l = content_list.to_dict(orient= "list")
for i in range(len(l['content'])):
  labels = ast.literal_eval(l['comparative'][i])
  for j in range(len(labels)-1,0,-1):
    if labels[j] == labels[j-1]:
      labels.pop(j)
  for j in range(len(labels)):
    dct['content'].append(l['content'][i])
    dct['comparative'].append(labels[j])
df_dct = pd.DataFrame(data=dct)
df_dct

ValueError: ignored

In [ ]:
train_text, valid_text, train_label, valid_label = train_test_split(content_list['content'], content_list['comparative'], test_size=0.05)

In [ ]:
train_encoding = tokenizer(train_text.to_list(), padding =True)
valid_encoding = tokenizer(valid_text.to_list(), padding =True)

#Class inherit pytorch Dataset

In [ ]:
class DatasetSequenceVLSP(torch.utils.data.Dataset):
  def __init__(self, encoding, labels):
        self.encoding = encoding
        self.labels = labels

  def __len__(self):
        return len(self.encoding['input_ids'])

  def __getitem__(self, idx):
        item = {k: torch.tensor(v[idx]) for k, v in self.encoding.items()}
        item["labels"] = torch.tensor([self.labels.iloc[idx]])
        return item

# def splitDataset(dataset, splitSize1, splitSize2, splitSize3, batchSize, shuffleDataset, randSeed):
#     datasetSize = len(dataset)
#     ind = list(range(datasetSize))
#     if (np.around(splitSize1 + splitSize2 + splitSize3) != 1):
#         print('WARNING: PROPORTION NOT SUM UP TO 1')
#     else:
#         splitInd = int(np.floor(splitSize1 * datasetSize))
#         splitInd2 = int(np.ceil(splitSize2 * datasetSize))
#     if (shuffleDataset):
#         np.random.seed(randSeed)
#         np.random.shuffle(ind)
#     splitSet1Ind, splitSet2Ind, splitSet3Ind = ind[: splitInd], ind[splitInd: splitInd + splitInd2], ind[splitInd + splitInd2:]
#     set1Sampler, set2Sampler, set3Sampler = (torch.utils.data.SubsetRandomSampler(splitSet1Ind),
#                                              torch.utils.data.SubsetRandomSampler(splitSet2Ind),
#                                              torch.utils.data.SubsetRandomSampler(splitSet3Ind))
#     return (torch.utils.data.DataLoader(dataset, batch_size = batchSize, sampler = set1Sampler),
#             torch.utils.data.DataLoader(dataset, batch_size = batchSize, sampler = set2Sampler),
#             torch.utils.data.DataLoader(dataset, batch_size = batchSize, sampler = set3Sampler))

In [ ]:
train_dataset = DatasetSequenceVLSP(train_encoding,train_label)
valid_dataset = DatasetSequenceVLSP(valid_encoding,valid_label)

In [ ]:
train_dataset.__len__(), valid_dataset.__len__()

(771, 41)

#Model setup

In [ ]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

In [ ]:
training_args = TrainingArguments(
    output_dir="/content/test_task1",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=5,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
)

trainer = Trainer(
    model=phobert,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset,
    compute_metrics=compute_metrics,
    # optimizers = (optim.Adam(phobert.parameters(),lr=0.05), None)
)

trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.283353,0.904306
2,No log,0.284359,0.913876
3,0.254700,0.258415,0.918660
4,0.254700,0.281114,0.933014
5,0.122400,0.286380,0.933014


TrainOutput(global_step=1240, training_loss=0.16761560593881913, metrics={'train_runtime': 564.4652, 'train_samples_per_second': 35.095, 'train_steps_per_second': 2.197, 'total_flos': 1333597911865800.0, 'train_loss': 0.16761560593881913, 'epoch': 5.0})

In [ ]:
trainer.save_model("test_bert_task1")

In [ ]:
from transformers import pipeline

model_test =  AutoModelForSequenceClassification.from_pretrained("/content/vncorenlp/test_bert_task1")
nlp = pipeline("sentiment-analysis", model=model_test, tokenizer=tokenizer)
example = "Điện_thoại A tốt hơn điện_thoại B"
ner_results = nlp(example)

In [ ]:
ner_results

[{'label': 'Comparative', 'score': 0.9720908999443054}]

In [ ]:
phobert.parameters()

<generator object Module.parameters at 0x784d94e3adc0>


In [ ]:
counter = [0 for i in range(len(id2label))]
for i in range(len(dct['content'])):
  counter[dct['label'][i]] += 1
counter

[0, 52, 251, 87, 4, 4, 372, 86, 19]